In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df1 = pd.read_csv('/kaggle/input/toronto-bikeshare-data/bikeshare2018/bikeshare2018/Bike Share Toronto Ridership_Q1 2018.csv')
df2 = pd.read_csv('/kaggle/input/toronto-bikeshare-data/bikeshare2018/bikeshare2018/Bike Share Toronto Ridership_Q2 2018.csv')
df3 = pd.read_csv('/kaggle/input/toronto-bikeshare-data/bikeshare2018/bikeshare2018/Bike Share Toronto Ridership_Q3 2018.csv')
df4 = pd.read_csv('/kaggle/input/toronto-bikeshare-data/bikeshare2018/bikeshare2018/Bike Share Toronto Ridership_Q4 2018.csv')

In [ ]:
df2018 = df1.copy()
df2018 = df2018.append([df2, df3, df4], sort=False)

In [ ]:
df2018.head()

In [ ]:
# prepare the dataset
df2018['time'] = pd.to_datetime(df2018.trip_start_time)
#df2018.index = df2018.time.dt.date
#df2018.index.name = 'index'

In [ ]:
df2018.index = df2018.time.dt.date
df2018.index.name = 'index'

In [ ]:
df2018 = df2018.dropna(subset=['from_station_name','to_station_name'])
df = df2018.copy()

In [ ]:
df

In [ ]:
station = list(set(list(df.from_station_name.values) + list(df.to_station_name.values)))
date = df.index.unique().values

In [ ]:
route = df.copy()
route = route[['from_station_name', 'to_station_name']]

**NOMOR 1**<br/>
kembali ke titik awal

In [ ]:
#two way, kembali ke titik awal
two_way = df.copy()
two_way = two_way[two_way.from_station_name == two_way.to_station_name]
# twoway = route.copy()
# twoway = twoway[twoway.from_station_name == twoway.to_station_name]
# oneway = route.copy()
# oneway = oneway[oneway.from_station_name != oneway.to_station_name]
two_way

In [ ]:
jmlh_two_way = len(two_way)
jmlh_semua = len(df)
jmlh_bukan_two_way = jmlh_semua - jmlh_two_way

persen_two_way = round((jmlh_two_way/jmlh_semua)*100,2)
persen_bukan_two_way = round((jmlh_bukan_two_way/jmlh_semua)*100,2)
print(persen_two_way," dan ",persen_bukan_two_way)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# The slices will be ordered and plotted counter-clockwise.
labels = [r'kembali ke titik Awal (2.98 %)', r'tidak kembali ke titik awal (97.02 %)']
sizes = [persen_two_way, persen_bukan_two_way]
colors = ['yellowgreen', 'lightskyblue']
patches, texts = plt.pie(sizes, colors=colors, startangle=90)
plt.legend(patches, labels, loc="best")
# Set aspect ratio to be equal so that pie is drawn as a circle.
plt.axis('equal')
plt.title("Persentase sepeda yang kembali ke titik awal")
plt.tight_layout()
plt.show()

In [ ]:
# two_way.user_type
two_way_annual = two_way.loc[two_way.user_type == 'Annual Member']
two_way_annual

In [ ]:
two_way_casual = two_way.loc[two_way.user_type == 'Casual Member']
two_way_casual

In [ ]:
len(two_way_casual)

In [ ]:
avg_cat = [len(two_way_annual), len(two_way_casual)]
index = ['Annual Member', 'Casual Member']
avg_calat= pd.DataFrame({'Jumlah Member': avg_cat}, index=index)
ax = avg_calat.plot.bar(rot=0, color='green', figsize=(13,8), title='Jumlah member yang mengembalikan sepeda ke titik awal', legend=True)
for p in ax.patches:
    ax.annotate(str(p.get_height()), (p.get_x() * 1.005, p.get_height() * 1.005))

**============Batas Nomor 1===============**

In [ ]:
route

In [ ]:
awal = route[['from_station_name']]
awal

In [ ]:
akhir = route[['to_station_name']]
akhir

In [ ]:
#ind = df2.user_id.isin(df1.user_id) & df1.user_id.isin(df2.user_id)

sama = (route.from_station_name) == (route.to_station_name)


In [ ]:
count=0
for s in sama:
    if s == True:
        count=count+1

In [ ]:
count

In [ ]:
cs=0
for r in sama:
    cs=cs+1
    
print(cs)

In [ ]:
round((57267/1922955)*100,2)

In [ ]:
twoway = route.copy()
twoway = twoway[twoway.from_station_name == twoway.to_station_name]
oneway = route.copy()
oneway = oneway[oneway.from_station_name != oneway.to_station_name]

**NOMOR 1, kembali ke titik awal**

In [ ]:
twoway

In [ ]:
oneway

In [ ]:
twoway_map = twoway.groupby('from_station_name').count().sort_values(by='to_station_name', ascending=False)
print('10 Stations with the highest number of two-way traveller')
twoway_map[:100]

In [ ]:
# mapping the number of outgoing bike from each station each day in 2017
outmap = pd.get_dummies(route.from_station_name).groupby('index').sum()
# mapping the number of incoming bike to each station each day in 2017
inmap = pd.get_dummies(route.to_station_name).groupby('index').sum()

In [ ]:
outmap.head(5) # number of bikes leaves the station

In [ ]:
inmap.head(5) # number of bikes entering the station

In [ ]:
print('number of station with enough bike to use next morning, aka number of bikes entering > number of bikes leaving the station')
((inmap - outmap)>=0).sum(axis=1)

In [ ]:
print('number of station with less bike to use next morning, or need a crew to return bikes back to station before next morning')
((inmap - outmap)<0).sum(axis=1)

In [ ]:
print('Station and the total number of days in 2018 where stations need more bikes to be returned by the crew every night')
((inmap - outmap)<0).sum(axis=0).sort_values(ascending=False)[:20]

In [ ]:
bike_minus = inmap - outmap # incoming bikes minus leaving bikes
bike_minus = np.absolute(bike_minus[bike_minus < 0]) # show only minus value
bike_minus

In [ ]:
print('20 Stations with the highest number of required returned bikes in a day')
np.max(bike_minus, axis=0).sort_values(ascending=False)[:20]

In [ ]:
f1=df['trip_duration_seconds']>86400
f1

In [ ]:
c_waktu=0
for s in f1:
    if s == True:
        c_waktu=c_waktu+1
        
print(c_waktu)

**Flow of Bike**

In [ ]:
import time 
import requests
import json
import networkx as nx
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [16,13]

In [ ]:
# get the stations information from https://tor.publicbikesystem.net
#req = requests.get('https://tor.publicbikesystem.net/ube/gbfs/v1/en/station_information')
# stations = json.loads(req.content)['data']['stations']
# stations = pd.DataFrame(stations)[['station_id', 'name', 'lat', 'lon']].astype({
#     'station_id': 'float64',
# })
stations = pd.read_csv('/kaggle/input/bikeshare-station/bikeshare_station.csv')
stations.head()

In [ ]:
root2018="/kaggle/input/toronto-bikeshare-data/bikeshare2018/"
files2018=[root2018+"bikeshare2018/Bike Share Toronto Ridership_Q1 2018.csv",
root2018+"bikeshare2018/Bike Share Toronto Ridership_Q2 2018.csv",
root2018+"bikeshare2018/Bike Share Toronto Ridership_Q3 2018.csv",
root2018+"bikeshare2018/Bike Share Toronto Ridership_Q4 2018.csv"]


In [ ]:
# load the data

first=True

for f in files2018:
    start_time = time.time()
    data = pd.read_csv(f,parse_dates = ['trip_start_time'],dayfirst=True)
    if first:
        df=data
        first=False
    else:
        df=df.append(data,sort=True)
    print("file {} -- {} seconds --".format(f,time.time() - start_time))        
del data , first

In [ ]:
# Hour of the day
df['hour'] = df['trip_start_time'].dt.hour
# Day of the week
df['dayofweek'] = df['trip_start_time'].dt.dayofweek
# Date
df['date'] = df['trip_start_time'].dt.floor('d')
# 2 stations in the dataset are not in the list of available stations, so map to the closest
df.loc[df['to_station_id']==7068,'to_station_id']=7399
df.loc[df['to_station_id']==7219,'to_station_id']=7148
df.loc[df['from_station_id']==7068,'from_station_id']=7399
df.loc[df['from_station_id']==7219,'from_station_id']=7148

In [ ]:
df.head()

In [ ]:
from sklearn.cluster import KMeans
X = stations[['lon', 'lat']].values
n_clusters_ = 20
kmeans = KMeans(n_clusters = n_clusters_, init ='k-means++')
kmeans.fit(X) # Compute k-means clustering.
labels = kmeans.fit_predict(X)

In [ ]:
# Coordinates of cluster centers.
centers = kmeans.cluster_centers_ 

# map each station with its region
stations['clusters']=labels
stations.index=stations['station_id']
cluster_map = stations['clusters'].to_dict()
df['from_station_c']=df['from_station_id'].map(cluster_map)
df['to_station_c']=df['to_station_id'].map(cluster_map)

In [ ]:
import matplotlib.image as mpimg
unique_labels = set(labels)
colors = [plt.cm.Spectral(each)
          for each in np.linspace(0, 1, len(unique_labels))]
BBox = (-79.4900,-79.2700,43.6100,43.7200)
# map_tor = mpimg.imread('map.png')
fig, ax = plt.subplots()
for k, col in zip(unique_labels, colors):
    if k == -1:
        # Black used for noise.
        col = [0, 0, 0, 1]

    class_member_mask = (labels == k)

    xy = X[class_member_mask]
    ax.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
             markeredgecolor='k', markersize=5)
    ax.plot(centers[k, 0], centers[k, 1], '^',markerfacecolor=tuple(col), markersize=18)
    ax.text(centers[k, 0], centers[k, 1], k,fontsize=12)
# ax.set_xlim(BBox[0],BBox[1])
# ax.set_ylim(BBox[2],BBox[3])
# ax.imshow(map_tor, zorder=0, extent = BBox, aspect= 'equal')
plt.show()

In [ ]:
# map_tor = mpimg.imread('map.png')
# ...
# ax.set_xlim(BBox[0],BBox[1])
# ax.set_ylim(BBox[2],BBox[3])
# ax.imshow(map_tor, zorder=0, extent = BBox, aspect= 'equal')

In [ ]:
##############################
# Count edges
##############################


df['region_count']=(df['from_station_c']<df['to_station_c']).astype(int)-(df['from_station_c']>df['to_station_c']).astype(int)
df['region_key'] = df[['from_station_c','to_station_c']].min(axis=1).astype('str') +'_'+ df[['from_station_c','to_station_c']].max(axis=1).astype('str')

df['station_count']=(df['from_station_id']<df['to_station_id']).astype(int)-(df['from_station_id']>df['to_station_id']).astype(int)
df['station_key'] = df[['from_station_id','to_station_id']].min(axis=1).astype('str') +'_'+ df[['from_station_id','to_station_id']].max(axis=1).astype('str')

df.describe(include=['O'])
df.columns

In [ ]:
import folium

In [ ]:
latitude = 43.651070
longitude = -79.347015
# create map and display it
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=12)

# display the map of San Francisco
toronto_map

In [ ]:
#koordinat stations

# instantiate a feature group for the incidents in the dataframe
incidents = folium.map.FeatureGroup()

# loop through the 100 crimes and add each to the incidents feature group
for lat, lng, in zip(stations.lat, stations.lon):
    incidents.add_child(
        folium.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='yellow',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6
        )
    )

# add incidents to map
toronto_map.add_child(incidents)

In [ ]:
import pandas as pd
import geopandas as gpd

import matplotlib.pyplot as plt
import plotly.graph_objects as go


In [ ]:
import networkx as nx
G = nx.DiGraph()
# nodeList = range(1,10)

# for i in range (0,len(nodeList)):
#     G.add_node(nodeList[i])

o_nodes = ['a','a','b','c','c','c','c','d','d','e','e','f','g','g','h','i','j','k']
d_nodes = ['h','c','c','a','g','b','d','e','c','d','f','d','c','i','a','g','k','j']

for i in range(0, len(o_nodes)):
    G.add_edge(o_nodes[i], d_nodes[i])  

nx.draw(G, with_labels = True)

In [ ]:
df.head(246)

In [ ]:
df.from_station_name.head(246)

In [ ]:
import networkx as nx
G = nx.DiGraph()
# nodeList = range(1,10)

# for i in range (0,len(nodeList)):
#     G.add_node(nodeList[i])
# awal_akhir = df['from_station_name'].head(246)
awal_akhir = pd.DataFrame({'from_station': df.from_station_name.head(246), 'to_station': df.to_station_name.head(246)}) 

# o_nodes = ['a','a','b','c','c','c','c','d','d','e','e','f','g','g','h','i','j','k']
# d_nodes = ['h','c','c','a','g','b','d','e','c','d','f','d','c','i','a','g','k','j']

o_nodes = awal_akhir['from_station']
d_nodes = awal_akhir['to_station']

for i in range(0, len(o_nodes)):
    G.add_edge(o_nodes[i], d_nodes[i])  

nx.draw(G, with_labels = True)

In [ ]:
import networkx as nx
G = nx.DiGraph()
# nodeList = range(1,10)

# for i in range (0,len(nodeList)):
#     G.add_node(nodeList[i])
# awal_akhir = df['from_station_name'].head(246)
awal_akhir = pd.DataFrame({'from_station': df.from_station_name.head(246), 'to_station': df.to_station_name.head(246)}) 

# o_nodes = ['a','a','b','c','c','c','c','d','d','e','e','f','g','g','h','i','j','k']
# d_nodes = ['h','c','c','a','g','b','d','e','c','d','f','d','c','i','a','g','k','j']

o_nodes = awal_akhir['from_station']
d_nodes = awal_akhir['to_station']

for i in range(0, len(o_nodes)):
    G.add_edge(o_nodes[i], d_nodes[i])  

nx.draw(G, with_labels = True)

In [ ]:
list(nx.simple_cycles(G))

In [ ]:
len(list(nx.simple_cycles(G)))

In [ ]:
nx.find_cycle(G, orientation="original")

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

# Create Directed Graph
G=nx.DiGraph()

# Add a list of nodes:
# awal_akhir = pd.DataFrame({'from_station': df.from_station_name.head(246), 'to_station': df.to_station_name.head(246)})
# station_satu_hari = list(set(list(awal_akhir.from_station.values) + list(awal_akhir.to_station.values)))
G.add_nodes_from(["a","b","c","d","e"])

# Add a list of edges:
# G.add_edges_from([("a","b"),("b","c"), ("c","a"), ("b","d"), ("d","e"), ("e","a")])
G.add_edges_from([("a","b")])
G.add_edges_from([("b","c")])
G.add_edges_from([("c","a")])
G.add_edges_from([("b","d")])
G.add_edges_from([("d","e")])
G.add_edges_from([("e","a")])

#Return a list of cycles described as a list o nodes
len(list(nx.simple_cycles(G)))

In [ ]:
# station_satu_hari[0:2]

# # Create Directed Graph
# G=nx.DiGraph()

# # Add a list of nodes:
# # awal_akhir = pd.DataFrame({'from_station': df.from_station_name.head(246), 'to_station': df.to_station_name.head(246)})
# # station_satu_hari = list(set(list(awal_akhir.from_station.values) + list(awal_akhir.to_station.values)))
# G.add_nodes_from(list(set(list(awal_akhir.from_station.values) + list(awal_akhir.to_station.values))))
# # hub=["a","b","c","d","e"]

# # Add a list of edges:
# # G.add_edges_from([("a","b"),("b","c"), ("c","a"), ("b","d"), ("d","e"), ("e","a")])


# G.add_edges_from([(list(awal_akhir.from_station))])

# #Return a list of cycles described as a list o nodes
# len(list(nx.simple_cycles(G)))

In [ ]:
awal_akhir.head()

In [ ]:
G = nx.DiGraph()
# nodeList = range(1,10)

# for i in range (0,len(nodeList)):
#     G.add_node(nodeList[i])
# awal_akhir = df['from_station_name'].head(246)
# awal_akhir = pd.DataFrame({'from_station': df.from_station_name.head(246), 'to_station': df.to_station_name.head(246)}) 

o_nodes = [1,2,3,2,7,9,8]
d_nodes = [2,4,5,5,9,10,7]

for i in range(0, len(o_nodes)):
    G.add_edge(o_nodes[i], d_nodes[i])  

nx.draw(G, with_labels = True)


In [ ]:
G.number_of_edges()

In [ ]:
G = nx.DiGraph([(0, 1), (0, 2), (1, 2)])
try:
    nx.find_cycle(G, orientation="original")
except:
    pass

list(nx.find_cycle(G, orientation="ignore"))


In [ ]:
# function to calculate the number of triangles in a simple 
# directed/undirected graph. 
# isDirected is true if the graph is directed, its false otherwise 
def countTriangle(g, isDirected): 
    nodes = len(g) 
    count_Triangle = 0 #Initialize result 
    # Consider every possible triplet of edges in graph 
    for i in range(nodes): 
        for j in range(nodes): 
            for k in range(nodes): 
                # check the triplet if it satisfies the condition 
                if( i!=j and i !=k and j !=k and 
                        g[i][j] and g[j][k] and g[k][i]): 
                    count_Triangle += 1
    # if graph is directed , division is done by 3 
    # else division by 6 is done 
    return count_Triangle/3 if isDirected else count_Triangle/6
  
# Create adjacency matrix of an undirected graph 
graph = [[0, 1, 1, 0], 
       [1, 0, 1, 1], 
       [1, 1, 0, 1], 
       [0, 1, 1, 0 ]] 
# Create adjacency matrix of a directed graph 
digraph = [ [0, 0, 1, 0], 
            [1, 0, 0, 1], 
            [0, 1, 0, 0], 
            [0, 0, 1, 0 ]] 
  
print ("The Number of triangles in undirected graph : %d" %countTriangle(graph, False)) 
  
print ("The Number of triangles in directed graph : %d" %countTriangle(digraph, True)) 
  

In [ ]:
import networkx as nx
nodes = [0, 1, 2, 3, 4, 5]
edges = [(0,1), (1,0), (1, 0),(0, 1), (2, 3), (2, 3), (2, 3), (2, 3),
         (4, 1), (4, 1), (4, 1), (4, 1), (4, 1), (4, 1), (4, 5), (5, 0)]
G = nx.MultiDiGraph()
G.add_nodes_from(nodes)
G.add_edges_from(edges)

from collections import Counter
width_dict = Counter(G.edges())
edge_width = [ (u, v, {'width': value}) 
              for ((u, v), value) in width_dict.items()]

G_new = nx.DiGraph()
G_new.add_edges_from(edge_width)

edges = G_new.edges()
weights = [G_new[u][v]['width'] for u,v in edges] 
nx.draw(G_new, edges=edges, width=weights)